구글 제미나이 API에서 매개변수는 GenerationConfig 객체를 통해 설정됩니다. GenerationConfig 객체에는 모델의 응답 수인 candidate_count를 포함하여 총 6개의 매개변수가 있습니다.

추론 시점(실행 시점)에는 그렇게 만들어진 모델의 출력값을 조정함으로써 언어모델이 더욱 다양하게 반응하도록 돕습니다. 학습 시점에 업데이트되는 가중치(weights)와 출력값을 조정하는 설정값 모두 매개변수라고 부르지만, 이 책에서는 특별한 경우가 아니면 두 번째 의미로 매개변수라는 용어를 사용합니다.

In [2]:
# single_turn
import google.generativeai as genai
import os

key_path = '/Users/jaesolshin/key/gemini_key.txt'
apikey = open(key_path, 'r', encoding='utf-8').read()
genai.configure(api_key=apikey)
model = genai.GenerativeModel('gemini-1.5-flash') # GenerativeModel 객체를 생성

response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.") # model의 generate_content 메서드를 통해 문자열로 메시지를 보내고 응답을 받습니다.
response.text

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'인공 지능(AI)은 컴퓨터가 인간과 같은 방식으로 생각하고, 배우고, 문제를 해결하도록 만드는 기술입니다. \n'

#  candidate_count

1장에서 다루었던 응답 후보(Candidate) 수를 설정하는 매개변수입니다. 현재는 기본값인 1만 허용되므로 추후 구글에서 응답 후보 수를 늘리기 전까지는 별도의 설정이 불필요합니다. 만일 1이 아닌 값을 설정하면 다음과 같이 오류가 발생합니다.

In [3]:
generation_config = genai.GenerationConfig(candidate_count=2)
model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)
response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")
print(response.text)
print(f"canidate 생성 건수: {len(response.candidates)}")

InvalidArgument: 400 Multiple candidates is not enabled for models/gemini-1.5-flash

# stop_sequences

언어모델이 언어를 생성하다가 stop_sequences에 있는 문자열을 만나면 생성을 중단합니다. 민감한 어휘의 등장을 막거나, 응답 길이를 제한할 때 유용하게 사용할 수 있습니다. 

다음은 마침표나 느낌표가 등장하면 언어 생성을 중지시키는 코드입니다. 출력 결과에서 알 수 있듯이 온점이 나타나자 언어 생성을 멈추었으며, 결과 반환 시 온점까지도 제외되었습니다. 

참고로, stop_sequences는 최대 5개까지 설정할 수 있으며, 초과 시 InvalidArgument 오류가 발생합니다.

In [4]:
# stop_sequneces.py
import google.generativeai as genai

generation_config = genai.GenerationConfig(stop_sequences=[". ","! "])
model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)
response = model.generate_content("인공지능에 대해 설명하세요.")

print(response.text)

## 인공지능: 인간의 지능을 모방하는 기술

**인공지능(AI)**은 컴퓨터 과학의 한 분야로, 인간의 지능을 모방하는 시스템을 만드는 것을 목표로 합니다


# max_output_tokens

제미나이 프로 모델은 영어의 경우 토큰 1개로 약 4글자를 표현합니다. 이에 비해 한글은 토큰 1개로 대략 1.5자를 표현합니다.

구글 제미나이 SDK에서 max_output_tokens는 모델이 생성하는 메시지가 최대 토큰 수를 넘지 않도록 제어하는 매개변수입니다.

In [9]:
# max_tokens.py
import google.generativeai as genai

generation_config = genai.GenerationConfig(max_output_tokens=10)
model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)
user_message = "인공지능에 대해 한 문장으로 설명하세요."
response = model.generate_content(user_message)

print('response : ', response.text)
print('finish reason : ', response._result.candidates[0].finish_reason.name)

response :  인공 지능(AI)은 인간
finish reason :  MAX_TOKENS


# temperature

0~2

In [16]:
# temperature.py
import google.generativeai as genai
model = genai.GenerativeModel('gemini-pro')
user_message = "겨울에 대한 짧은 시를 20자 이내로 지으세요."

print("\ntemperature=0:")
generation_config = genai.GenerationConfig(temperature=0)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')

print("\ntemperature=0.5:")
generation_config = genai.GenerationConfig(temperature=0.5)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')

print("\ntemperature=1:")
generation_config = genai.GenerationConfig(temperature=1)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')


temperature=0:
눈 내리고, 추위 몰아치고,
겨울의 품에 안겨.
눈 내리고, 추위 몰아치고,
겨울의 품에 안겨.
눈 내리고, 추위 몰아치고,
겨울의 품에 안겨.

temperature=0.5:
눈 내리고, 추위 몰아치네,
겨울이 왔네.
눈의 흰 캔버스에
겨울이 그림 그려


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

# top_p

temperature가 확률분포를 조정하는 매개변수라면, top_p는 확률분포 내에서 선택할 단어의 범위를 결정하는 매개변수

누적확률이 top_p가 될때까지의 단어만을 가지고 문장을 생성한다. top_p=0으로 설정한다면 확률분포 중 가장 높은 확률의 단어만 선택.

주의 ) temperature=0이나 top_p=0으로 설정했다고 해서 매번 완전히 동일한 결과만 생성하는 것은 아닙니다. 거대언어모델은 클라우드 환경에서 여러 노드의 리소스를 병렬적으로 사용하는 방식으로 수많은 연산을 수행할 뿐만 아니라, 확률분포에 동일한 확률값을 갖는 단어들도 나타날 수 있으므로 항상 같은 결과를 보장하기는 어렵습니다.

In [17]:
# top_p.py
import google.generativeai as genai
model = genai.GenerativeModel('gemini-pro')
user_message = "겨울에 대한 짧은 시를 20자 이내로 지으세요."

print("\ntop_p=0:")
generation_config = genai.GenerationConfig(top_p=0)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')

print("\ntop_p=1:")
generation_config = genai.GenerationConfig(top_p=1)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')


NameError: name 'count_word_occurrences' is not defined

# top_k

기본값은 64.  

k개의 단어가 선택된 이후의 동작은 top_p와 같다.

k개의 단어가 선택되는 과정에서 단어 간의 확률 편차가 고려되지 않으므로, 건드리지 않는 것이 좋다.

# 초기값

매개변수의 초깃값은 다음 명령으로 확인할 수 있습니다.

import google.generativeai as genai
print(genai.get_model("models/gemini-1.5-flash"))


In [18]:
import google.generativeai as genai
print(genai.get_model("models/gemini-1.5-flash"))

Model(name='models/gemini-1.5-flash',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash',
      description='Fast and versatile multimodal model for scaling across diverse tasks',
      input_token_limit=1000000,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)


# 안전성 점검체계

HARASSMENT (괴롭힘)	- 성별, 성적지향, 종교, 인종 등 보호받는 개인의 특성에 대해 부정적이거나 해로운 언급을 하는 행위

HATE SPEECH (증오심 표현) - 무례하거나 존중하지 않는 태도 또는 저속한 발언

SEXUAL EXPLICITY (음란물) - 성행위 또는 성적으로 노골적인 내용

DANGEROUS (위해성) - 해로운 행위를 야기하는 내용


각각의 카테고리에 대해 4가지의 위반 클래스를 제공 - NEGLIGIBLE, LOW, MEDIUM, HIGH

각각의 위반 확률에 대해서는 4단계의 기준점을 설정할 수 있다 - BLOCK_NONE, BLOCK_ONLY_HIGH, BLOCK_MEDIUM_AND_ABOVE, BLOCK_LOW_AND_ABOVE

초기값은 BLOCK_MEDIUM_AND_ABOVE.

In [22]:
import google.generativeai as genai
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("당신은 뛰어난 연극 배우입니다. 화난 대사를 읊어보세요.")  

print(response._result)


candidates {
  finish_reason: SAFETY
  index: 0
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: HIGH
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 24
  total_token_count: 24
}



In [23]:
import google.generativeai as genai

# 모델을 호출하는 부분. gemini-1.5-flash라는 모델을 이용하여 텍스트를 생성함.
model = genai.GenerativeModel('gemini-1.5-flash')

# 안전성 필터 설정 (BLOCK_NONE) - 위반 수준에 관계없이 콘텐츠가 차단되지 않음.
response = model.generate_content(
    "당신은 뛰어난 연극 배우입니다. 화난 대사를 읊어보세요.",
    safety_settings={
        "block_level": "BLOCK_NONE"
    }
)

# 생성된 콘텐츠를 출력.
print(response._result)

KeyError: 'block_level'